In [1]:
#all packages which will be used
import numpy as np
import pandas as pd
from scipy.io import loadmat
import time
import random

#plot
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from matplotlib.axes import Axes

#sklearn
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
#from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_score, recall_score,f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.kernel_approximation import RBFSampler #change the linear kernel into RBF
#to plot more than 2 dimensions 
from mlxtend.plotting import plot_decision_regions

#scipy
from scipy.stats import chisquare
from scipy.stats import chi2
from scipy.spatial import distance

#good performance classifier
from xgboost import XGBClassifier

#Tensorflow
import tensorflow as tf


#math
from math import log

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



In [2]:
train_set=pd.read_csv("C:\\Users\\hasee\\workspace\\lab\\patRecDat\\forStudents\\medData\\train_set.csv")
X_val=pd.read_csv("C:\\Users\\hasee\\workspace\\lab\\patRecDat\\forStudents\\medData\\val_feature.csv")
y_val=pd.read_csv("C:\\Users\\hasee\\workspace\\lab\\patRecDat\\forStudents\\medData\\val_label.csv")
X_train=train_set.drop(["label"],axis=1)
y_train=train_set["label"]

In [3]:
def tf_maha(data):
    '''
    Compute mahalanobis distance of every point to the mean
    '''
    num_obs, num_vars = data.shape
    with tf.Session() as sess:
        dd = tf.placeholder(tf.float64,[num_obs,num_vars])
        means = tf.reduce_mean(dd, reduction_indices=[0])
        diffs = tf.subtract(dd,means)
        cov = (1/float(num_obs-1)) * tf.matmul(tf.transpose(diffs),diffs)
        vivi = tf.matrix_inverse(cov)
        dt = tf.transpose(diffs)
        ds = diffs
        M1 = tf.matmul(ds,vivi)
        M2 = tf.matmul(M1,dt)
        z = tf.stack([M2[i,i] for i in range(num_obs)]) #tf,pack has been renamed as stack
        output = tf.sqrt(z)
        ans = sess.run([output],feed_dict = {dd:data})
    return ans

In [4]:
def maha_tensor(X,y,cov):
    #if use tf.variable need initial value
    uu=tf.placeholder(tf.float64,[X.shape[0],X.shape[1]])
    vv=tf.placeholder(tf.float64,[y.shape[0],y.shape[1]])
    V=tf.placeholder(tf.float64,[X.shape[1],y.shape[1]])
    
    
    diff=tf.subtract(uu,vv)
    VI=tf.matrix_inverse(V)
    maha=tf.sqrt(tf.matmul(tf.matmul(diff,VI),diff))
    
    with tf.Session() as sess:
        maha_dist=sess.run(maha,feed_dict={uu:X,vv:y,V:cov})
    return maha_dist
    

In [ ]:
tf_maha(X_train)

In [ ]:
#implement the maha_tensor function
tf.reset_default_graph()
mean=np.array(X_train.mean()).reshape(1,-1)
cov=np.matrix(X_train.cov())
#maha_tensor(X_train,mean,cov)
dis=distance.cdist(X_train,mean,metric='mahalanobis',V=cov).ravel()

In [ ]:
start=time.time()
dis=np.sqrt(np.dot(np.dot(np.subtract(X_train.loc[:10000],mean),cov.I),np.subtract(X_train.loc[:10000],mean).T))
print(time.time()-start)